In [1]:
from pathlib import Path
import pandas as pd
import csv

In [2]:
datapath = Path('../../data/raw/nz_raw_data')

In [3]:
file = 'NZ 0321_Universities_Workforce_Data_gender_ethnicity 2000-2017.xlsx'

In [4]:
xl = pd.ExcelFile(datapath/file)

In [5]:
ethnicity = pd.read_excel(xl, 'Ethnic Group', index_col=[0,1], 
                          skiprows=4, usecols=[1,2,3,4,5,6,8,9])
ethnicity

2012  2013  2014   2015  \
Provider                          Ethnic Group                               
University of Auckland            European         1365  1660  1620   1890   
                                  Māori             115   185   155    185   
                                  Pacific Peoples    45    80    65    115   
                                  Asian             240   345   340    725   
                                  Other             410   345   375    470   
                                  Unknown             0     0     0      0   
                                  Total            2180  2615  2560   3385   
University of Waikato             European          490   465   455    440   
                                  Māori              60    60    55     55   
                                  Pacific Peoples    10    10    10     10   
                                  Asian              40    40    40     45   
                                  Other              40    40    50     65   
                                  Unknown            35    30    30     35   
                                  Total             670   650   640    645   
Massey University                 European          980   970  1005   1020   
                                  Māori              50    55    50     55   
                                  Pacific Peoples    10    15    10     10   
                                  Asian             115   120   120    130   
                                  Other              70    75    80     80   
                                  Unknown           150   125   120    110   
                                  Total            1380  1355  1390   1405   
Victoria University of Wellington European          495   560   645    770   
                                  Māori              30    45    60     50   
                                  Pacific Peoples    15    15    10     15   
                                  Asian              70    70    75     95   
                                  Other              20    25    35     40   
                                  Unknown           335   330   325    315   
                                  Total             935  1005  1095   1115   
University of Canterbury          European          495   150   440    425   
                                  Māori              25    20    20     25   
...                                                 ...   ...   ...    ...   
                                  Unknown            95   100   100    110   
                                  Total             725   680   655    645   
Lincoln University                European          160   175   170    150   
                                  Māori              10     5     5     10   
                                  Pacific Peoples     5     5     5      0   
                                  Asian              10    10    15     15   
                                  Other              15    15    15     30   
                                  Unknown            35    25    30     20   
                                  Total             235   230   240    225   
University of Otago               European          855   845   845    955   
                                  Māori              50    50    50     60   
                                  Pacific Peoples    10    15    15     15   
                                  Asian              55    60    60     75   
                                  Other              55    75    75     80   
                                  Unknown           510   525   555    420   
                                  Total            1470  1490  1515   1510   
Auckland University of Technology European          690   705   730    740   
                                  Māori              40    50    50     60   
                                  Pacific Peoples    20    35    35     3

In [7]:
coki_format = []
for year in range(2012, 2016):
    for uni,ethn in ethnicity.index.values:
        d = {'grid' : 'none',
                'uni_name' : uni,
                'year' : year,
                'country' : 'New Zealand',
                'region' : 'Oceania',
                'subregion' : 'Australasia',
                'data_name' : 'diversity_nz_doe_{}_august_snapshot_num'.format(ethn.lower()),
                'data_value' : ethnicity.loc[(uni,ethn)][year]
                }
        coki_format.append(d)

for year in [2016,2017]:
    for uni,ethn in ethnicity.index.values:
        d = {'grid' : 'none',
                'uni_name' : uni,
                'year' : year,
                'country' : 'New Zealand',
                'region' : 'Oceania',
                'subregion' : 'Australasia',
                'data_name' : 'diversity_nz_doe_{}_year_end_num'.format(ethn.lower()),
                'data_value' : ethnicity.loc[(uni,ethn)][year]
                }
        coki_format.append(d)        
        
nz_doe_df = pd.DataFrame(coki_format)

In [9]:
gender_num = pd.read_excel(xl, 'Gender x Full-Part Time', index_col=[0,1,2], 
                          skiprows=4, usecols="B:AT,AV:BA")
gender_num

Females  \
Number of academic staff employed or contracted... Provider                          Tenure               
Number of academic staff                           University of Auckland            Part-time      505   
                                                                                     Full-time      355   
                                                                                     Total          860   
                                                   University of Waikato             Part-time       90   
                                                                                     Full-time      260   
                                                                                     Total          350   
                                                   Massey University                 Part-time      145   
                                                                                     Full-time      355   
                                                                                     Total          500   
                                                   Victoria University of Wellington Part-time       35   
                                                                                     Full-time      180   
                                                                                     Total          215   
                                                   University of Canterbury          Part-time      300   
                                                                                     Full-time      145   
                                                                                     Total          440   
                                                   Lincoln University                Part-time       25   
                                                                                     Full-time       40   
                                                                                     Total           65   
                                                   University of Otago               Part-time      290   
                                                                                     Full-time      255   
                                                                                     Total          545   
                                                   Auckland University of Technology Part-time      235   
                                                                                     Full-time      345   
                                                                                     Total          580   
                                                   Total                             Part-time     1620   
                                                                                     Full-time     1930   
                                                                                     Total         3550   
Full-time Equivalent staff (FTE)                   University of Auckland            Part-time      145   
                                                                                     Full-time      355   
                                                                                     Total          500   
                                                   University of Waikato             Part-time       40   
                                                                                     Full-time      260   
                                                                                     Total          300   
                                                   Massey University                 Part-time       85   
                                                                                     Full-time      355   
                                                                                     Total          440   
                              

In [12]:
coki_format = []
for year in range(2002, 2018):
    for dataitem in gender_num.index.values:
        calc = {'Number of academic staff' : 'num',
                'Full-time Equivalent staff (FTE)' : 'fte'}[dataitem[0]]
        uni = dataitem[1]
        tenure = dataitem[2]
        for group in ['Females', 'Males', 'Total']:
            if year!=2002:
                group_key = group+'.'+str(year-2002)
            else:
                group_key = group
            d = {'grid' : 'none',
                'uni_name' : uni,
                'year' : year,
                'country' : 'New Zealand',
                'region' : 'Oceania',
                'subregion' : 'Australasia',
                'data_name' : 'diversity_nz_doe_academic_{}_{}_{}'.format(group.lower(),
                                                                                  tenure.replace('-', '').lower(),
                                                                                  calc),
                                                                                  
                'data_value' : gender_num.loc[dataitem][group_key]
                }
            coki_format.append(d)

coki_gender_df = pd.DataFrame(coki_format)
nz_doe_df.append(coki_format)

In [31]:
level_ethn_gender_partfull_num = pd.read_excel(xl, 'Ethnic x Full-Part Time', index_col=[0,1,2], 
                          skiprows=6, usecols="B:AN,AP:BG")
level_ethn_gender_partfull_num

Females  \
Provider               Staff Type/Group                         Ethnic Group               
University of Auckland Professors/Deans                         European              15   
                                                                Māori                  0   
                                                                Pacific Peoples        0   
                                                                Asian                  0   
                                                                Other                  0   
                                                                Total                 15   
                       Assoc.Professors/Heads of Department     European              15   
                                                                Māori                  0   
                                                                Pacific Peoples        0   
                                                                Asian                  5   
                                                                Other                  5   
                                                                Total                 25   
                       Senior Lecturers/Principal Lecturers     European              55   
                                                                Māori                 10   
                                                                Pacific Peoples        0   
                                                                Asian                  5   
                                                                Other                 10   
                                                                Total                 80   
                       Lecturers/Senior tutors/Tutors           European              20   
                                                                Māori                  5   
                                                                Pacific Peoples        0   
                                                                Asian                  0   
                                                                Other                 10   
                                                                Total                 35   
                       Other academic staff/Tutorial assistants European             185   
                                                                Māori                 20   
                                                                Pacific Peoples       10   
                                                                Asian                 55   
                                                                Other                 55   
                                                                Total                325   
...                                                                                  ...   
Total                  Assoc.Professors/Heads of Department     European              35   
                                                                Māori                  0   
                                                                Pacific Peoples        0   
                                                                Asian                  5   
                                                                Other                  5   
                                                                Total                 55   
                       Senior Lecturers/Principal Lecturers     European             210   
                                                                Māori                 10   
                                                                Pacific Peoples        5   
                                                                Asian                 20   
                                                                Other                 15   
                                 

In [40]:
level_ethn_gender_partfull_num = pd.read_excel(xl, 'Ethnic x Full-Part Time', index_col=[0,1,2], 
                          skiprows=6, usecols="B:AN,AP:BG")
coki_format = []

for dataitem in level_ethn_gender_partfull_num.index.values:
    uni = dataitem[0]
    seniority = dataitem[1]
    ethnicity = dataitem[2]
    for year in range(2012, 2018):    
        for a, fullpart in enumerate(['fulltime', 'parttime', 'totaltime']):
            for group in enumerate(['Females', 'Males', 'Total']):
                colnum = (year-2012)+a
                
                if colnum == 0:
                    group_key = group[1]
                else:
                    group_key = group[1]+'.'+str(colnum)
                d = {'grid' : 'none',
                'uni_name' : uni,
                'year' : year,
                'country' : 'New Zealand',
                'region' : 'Oceania',
                'subregion' : 'Australasia',
                'data_name' : 'diversity_nz_doe_academic_{}_{}_{}_fte'.format(seniority.lower(),
                                                                                  ethnicity.lower(),
                                                                                  group[1].lower()),
                                                                                  
                'data_value' : level_ethn_gender_partfull_num.loc[dataitem][group_key]
                }
                coki_format.append(d)

level_ethn_gender_partfull_num_df = pd.DataFrame(coki_format)
level_ethn_gender_partfull_num_df
nz_doe_df.append(coki_format)

,country,data_name,data_value,grid,region,subregion,uni_name,year
0,New Zealand,diversity_nz_doe_academic_professors/deans_eur...,15,none,Oceania,Australasia,University of Auckland,2012
1,New Zealand,diversity_nz_doe_academic_professors/deans_eur...,40,none,Oceania,Australasia,University of Auckland,2012
2,New Zealand,diversity_nz_doe_academic_professors/deans_eur...,50,none,Oceania,Australasia,University of Auckland,2012
3,New Zealand,diversity_nz_doe_academic_professors/deans_eur...,30,none,Oceania,Australasia,University of Auckland,2012
4,New Zealand,diversity_nz_doe_academic_professors/deans_eur...,115,none,Oceania,Australasia,University of Auckland,2012
5,New Zealand,diversity_nz_doe_academic_professors/deans_eur...,150,none,Oceania,Australasia,University of Auckland,2012
6,New Zealand,diversity_nz_doe_academic_professors/deans_eur...,45,none,Oceania,Australasia,University of Auckland,2012
7,New Zealand,diversity_nz_doe_academic_professors/deans_eur...,155,none,Oceania,Australasia,University of Auckland,2012
8,New Zealand,diversity_nz_doe_academic_professors/deans_eur...,200,none,Oceania,Australasia,University of Auckland,2012
9,New Zealand,diversity_nz_doe_academic_professors/deans_eur...,30,none,Oceania,Australasia,University of Auckland,2013


In [34]:
level_ethn_gender_partfull_num.index.values[0]

('University of Auckland', 'Professors/Deans', 'European')

In [19]:
level_ethn_gender_num = pd.read_excel(xl, 'Staff type x Ethnic x Gender', index_col=[0,1,2], 
                          skiprows=5, usecols="B:D,E:P,R:W")
level_ethn_gender_num

Females  \
Provider               Staff Type/Group                         Ethnic Group               
University of Auckland Professors/Deans                         European              40   
                                                                Māori                  0   
                                                                Pacific Peoples        0   
                                                                Asian                  0   
                                                                Other                 10   
                       Assoc.Professors/Heads of Department     European              65   
                                                                Māori                  5   
                                                                Pacific Peoples        0   
                                                                Asian                  5   
                                                                Other                 10   
                       Senior Lecturers/Principal Lecturers     European             180   
                                                                Māori                 20   
                                                                Pacific Peoples       15   
                                                                Asian                 15   
                                                                Other                 35   
                       Lecturers/Senior tutors/Tutors           European              50   
                                                                Māori                 10   
                                                                Pacific Peoples        5   
                                                                Asian                 10   
                                                                Other                 25   
                       Other academic staff/Tutorial assistants European             140   
                                                                Māori                 15   
                                                                Pacific Peoples       10   
                                                                Asian                 50   
                                                                Other                 35   
                       Total                                    European             470   
                                                                Māori                 55   
                                                                Pacific Peoples       30   
                                                                Asian                 80   
                                                                Other                115   
...                                                                                  ...   
Total                  Professors/Deans                         European             130   
                                                                Māori                  5   
                                                                Pacific Peoples        0   
                                                                Asian                  5   
                                                                Other                 20   
                       Assoc.Professors/Heads of Department     European             225   
                                                                Māori                 15   
                                                                Pacific Peoples        0   
                                                                Asian                 10   
                                                                Other                 25   
                       Senior Lecturers/Principal Lecturers     European             840   
                                 

In [26]:
coki_format = []
for year in range(2012, 2018):
    for dataitem in level_ethn_gender_num.index.values:
        uni = dataitem[0]
        seniority = dataitem[1]
        ethnicity = dataitem[2]
        for group in ['Females', 'Males', 'Total']:
            if year!=2012:
                group_key = group+'.'+str(year-2012)
            else:
                group_key = group
            d = {'grid' : 'none',
                'uni_name' : uni,
                'year' : year,
                'country' : 'New Zealand',
                'region' : 'Oceania',
                'subregion' : 'Australasia',
                'data_name' : 'diversity_nz_doe_academic_{}_{}_{}_num'.format(seniority.lower(),
                                                                                  ethnicity.lower(),
                                                                                  group.lower()),
                                                                                  
                'data_value' : level_ethn_gender_num.loc[dataitem][group_key]
                }
            coki_format.append(d)

level_ethn_gender_num_df = pd.DataFrame(coki_format)
nz_doe_df.append(coki_format)

In [28]:
level_ethn_gender_fte = pd.read_excel(xl, 'Staff type x Ethnic x Gender', index_col=[0,1,2], 
                          skiprows=5, usecols="B:D,X:AI,AK:AP")
coki_format = []
for year in range(2012, 2018):
    for dataitem in level_ethn_gender_fte.index.values:
        uni = dataitem[0]
        seniority = dataitem[1]
        ethnicity = dataitem[2]
        for group in ['Females', 'Males', 'Total']:
            if year!=2012:
                group_key = group+'.'+str(year-2012)
            else:
                group_key = group
            d = {'grid' : 'none',
                'uni_name' : uni,
                'year' : year,
                'country' : 'New Zealand',
                'region' : 'Oceania',
                'subregion' : 'Australasia',
                'data_name' : 'diversity_nz_doe_academic_{}_{}_{}_fte'.format(seniority.lower(),
                                                                                  ethnicity.lower(),
                                                                                  group.lower()),
                                                                                  
                'data_value' : level_ethn_gender_fte.loc[dataitem][group_key]
                }
            coki_format.append(d)

level_ethn_gender_fte_df = pd.DataFrame(coki_format)
nz_doe_df.append(coki_format)

In [ ]:
nz_doe_df